In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

100%|██████████| 25.7M/25.7M [00:02<00:00, 12.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [2]:
import shutil
shutil.move('/root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1' , '/content')

'/content/1'

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [4]:
dataset = '/content/1/IMDB Dataset.csv'
df = pd.read_csv(dataset)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.isnull().value_counts()

,,count
review,sentiment,
False,False,50000


In [6]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [7]:
positive = 0
negative = 0
other = 0
for value in df['sentiment']:
    if value == 'positive':
        positive+=1
    elif value == 'negative':
        negative+=1
    else:
        other+=1
positive , negative , other

(25000, 25000, 0)

In [8]:
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

In [9]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
x = df['review']
y = df['sentiment']

In [11]:
x_train , y_train , x_test , y_test = train_test_split(x,y , test_size = 0.2 , random_state=42)

In [12]:
train_data , test_data = train_test_split(df , test_size = 0.2 , random_state=42)

In [13]:
print(train_data.shape)

(40000, 2)


In [14]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']) , maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']) , maxlen = 200)


In [15]:
print(x_train)
print(x_train.shape)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
(40000, 200)


In [16]:
print(x_test)
print(x_test.shape)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]
(10000, 200)


In [17]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [18]:
print(y_train.shape)
print(y_test.shape)

(40000,)
(10000,)


In [19]:
model = Sequential()
model.add(Embedding(input_dim = 5000 , output_dim = 128 , input_length = 200 ))
model.add(LSTM(128 , dropout=0.2 , recurrent_dropout=0.2))
model.add(Dense(1 , activation = 'sigmoid'))
model.build((None, 200))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 392ms/step - accuracy: 0.7067 - loss: 0.5495 - val_accuracy: 0.8447 - val_loss: 0.3634
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 404ms/step - accuracy: 0.8439 - loss: 0.3690 - val_accuracy: 0.7749 - val_loss: 0.4631
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 407ms/step - accuracy: 0.8544 - loss: 0.3463 - val_accuracy: 0.8706 - val_loss: 0.3161
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 262s 408ms/step - accuracy: 0.8929 - loss: 0.2675 - val_accuracy: 0.8720 - val_loss: 0.3226
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 254s 391ms/step - accuracy: 0.9033 - loss: 0.2440 - val_accuracy: 0.8798 - val_loss: 0.3113
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 404ms/step - accuracy: 0.9238 - loss: 0.1968 - val_accuracy: 0.8666 - val_loss: 0.3179
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 252s 385ms/step - accuracy: 0.9308 - loss: 0.1790 - val_accuracy: 0.8736 - val_loss: 0.3430
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 211s 402ms/step - accuracy: 0.9390 -

In [23]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 116ms/step - accuracy: 0.8709 - loss: 0.6464
Test Loss: 0.6443732380867004
Test Accuracy: 0.8726999759674072


In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [25]:
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
The sentiment of the review is: negative


In [26]:
model.save('sentiment_analysis.h5')